In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='dataset/data.yaml', epochs=30, imgsz=640)


# Evaluate the model
metrics = model.val()


In [ ]:
model.val(data='dataset/data.yaml')

DETECT WITH TEST IMAGES

In [ ]:
from ultralytics import YOLO
import os

# Function to perform inference and display results
def run_inference(model_path, image_filename, dataset_dir):
    # Construct the full image path using the dataset directory and image filename
    image_path = os.path.join(dataset_dir, image_filename)
    
    # Load the model with the trained weights
    model = YOLO(model_path)
    
    # Perform inference on the specified image
    results = model(image_path, save=True)
    
    # Display results (if it's a list, access the first item)
    results[0].show()

# Specify the path to the trained weights
model_path = 'best.pt'

# Specify the image filename you want to run inference on
image_filename = 'IMG_1131_000290_jpg.rf.806bd79d72734eb31740e90c72b0addc.jpg'

# Specify the dataset directory
dataset_dir = 'dataset/test/images'

# Run inference with the specified model and image
run_inference(model_path, image_filename, dataset_dir)


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Specify the path to the image
image_path = 'dataset/train/images/IMG_1128_000025_jpg.rf.112af7e57a332d65fae4bc2507f4e958.jpg'

# Load the image using OpenCV
image = cv2.imread(image_path)

# Check if the image was loaded correctly
if image is None:
    print(f"Error: Image at {image_path} could not be loaded. Please check the path.")
else:
    # Convert BGR image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image using matplotlib
    plt.imshow(image)
    plt.axis('off')  # Turn off axis labels
    plt.show()


DETECT WITH VIDEO

In [ ]:
import cv2
from ultralytics import YOLO

# Function to perform inference on a video
def detect_in_video(model_path, video_path, output_video_path):
    # Load the model with the trained weights
    model = YOLO(model_path)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video at {video_path}")
        return
    
    # Get the video frame width, height, and frames per second (fps)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create a VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # or use 'MP4V' for .mp4
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        # Read each frame from the video
        ret, frame = cap.read()
        
        if not ret:
            break  # End of video

        # Perform inference on the current frame
        results = model(frame)

        # Display the results on the frame (bounding boxes and labels)
        annotated_frame = results[0].plot()  # Get the frame with bounding boxes
        
        # Write the frame to the output video file
        out.write(annotated_frame)

        # Display the frame
        cv2.imshow('Video Detection', annotated_frame)

        # Press 'q' to quit the video window early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video objects and close windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Specify the path to the trained model
model_path = 'best.pt'

# Specify the input video file
video_path = 'slow-video.mp4'

# Specify the output video file (where you want to save the annotated video)
output_video_path = 'output_video.avi'

# Run the detection on the video
detect_in_video(model_path, video_path, output_video_path)
